In [4]:
import torch
import torch.nn as nn
from torch_geometric.loader import DataLoader
from networks import FractalNet, FractalNetShared, Net, GNN_no_rel, GNN, TransformerNet
from subgraph import Graph_to_Subgraph
from train import train_model, get_qm9

In [5]:
# GLOBAL VARIABLES FOR THE EXPERIMENT
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 55
batch_size = 32
node_features = 5
# data related (x and y)
Z_ONE_HOT_DIM = 5
LABEL_INDEX = 7
EDGE_ATTR_DIM = 4
edge_features = 0
hidden_features = 64
out_features = 1

# TRAINING SHARED PARAMETERS FRACTAL NET

In [3]:
model_name = 'FractalNetShared'
model = FractalNetShared(node_features,
                           edge_features,
                           hidden_features,
                           out_features,
                           depth=1,
                           pool='add').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)
train, valid, test = get_qm9("data/qm9",
                             device=device,
                             LABEL_INDEX = LABEL_INDEX,
                             transform=Graph_to_Subgraph())
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=32, shuffle=False)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

/home/tin/Documents/GitHub/FractalMessagePassing/train.py:23: UserWarning: Using non-standard permutation since permute.pt does not exist.
  warn("Using non-standard permutation since permute.pt does not exist.")


In [10]:
fractalnetshared_results = train_model(model, model_name, epochs, train_loader, valid_loader, test_loader, optimizer, criterion, scheduler, device, LABEL_INDEX, Z_ONE_HOT_DIM)

Total number of parameters: 25281


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

ValueError: Model name not recognized

# TRAINING FRACTAL NET

In [13]:
model_name = 'FractalNet'
model = FractalNet(node_features,
                   edge_features,
                   hidden_features,
                   out_features,
                   depth=3,
                   pool='add',
                   add_residual_skip=True).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)
train, valid, test = get_qm9("data/qm9",
                             device=device,
                             LABEL_INDEX=LABEL_INDEX,
                             transform=Graph_to_Subgraph())
train = train[:int(0.1 * len(train))]
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=32, shuffle=False)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

In [14]:
fractalnet_results = train_model(model, model_name, epochs, train_loader, valid_loader, test_loader, optimizer, criterion, scheduler, device, LABEL_INDEX, Z_ONE_HOT_DIM)

Total number of parameters: 298433


  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 0, Loss: 0.8165286079453774, Valid Loss: 0.3341747196718527


...

  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 3, Loss: 0.022421466380234262, Valid Loss: 0.021305969437977045


  0%|          | 0/313 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Training a Transformer Subgraph GNN

In [6]:
#device = 'cpu'
model_name = 'TransformerNet'
mode = 'transformer_10'
transformer_size = int(mode.split('_')[1])
model = TransformerNet(node_features+transformer_size, # this is needed as now we have more onehot encodings due to unique transformer nodes
                   edge_features,
                   hidden_features,
                   out_features,
                   depth=3,
                   pool='add',
                   add_residual_skip=True,
                   num_heads=4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)

train, valid, test = get_qm9("data/qm9",
                             device=device,
                             LABEL_INDEX=LABEL_INDEX,
                             transform=Graph_to_Subgraph(mode=mode))
# take only 10% of the training data
train = train[:int(0.1 * len(train))]
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

valid_loader = DataLoader(valid, batch_size=32, shuffle=False)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

/home/tin/Documents/GitHub/FractalMessagePassing/train.py:23: UserWarning: Using non-standard permutation since permute.pt does not exist.
  warn("Using non-standard permutation since permute.pt does not exist.")


In [7]:
fractalnet_transformer_results = train_model(model, model_name, epochs, train_loader, valid_loader, test_loader, optimizer, criterion, scheduler, device, LABEL_INDEX, Z_ONE_HOT_DIM)

Total number of parameters: 406593


  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (6237x256 and 64x256)

# TRAINING SAME NET AS FRACTAL BUT WITHOUT SUBNODES

In [5]:
model_name = 'Net'
model = Net(node_features,
            edge_features,
            hidden_features,
            out_features,
            depth=3,
            pool='add').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)
train, valid, test = get_qm9("data/qm9",
                             device=device,
                             LABEL_INDEX=LABEL_INDEX,
                             transform=None)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=32, shuffle=False)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

/home/tin/Documents/GitHub/FractalMessagePassing/train.py:23: UserWarning: Using non-standard permutation since permute.pt does not exist.
  warn("Using non-standard permutation since permute.pt does not exist.")


In [6]:
no_subnode_results = train_model(model, model_name, epochs, train_loader, valid_loader, test_loader, optimizer, criterion, scheduler, device, LABEL_INDEX, Z_ONE_HOT_DIM)

Total number of parameters: 74945


  0%|          | 0/55 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_index cpu
pos cpu
idx cpu
batch cpu
x cpu
z cpu
y cpu
ptr cpu
edge_attr cpu
edge_i

KeyboardInterrupt: 

# TRAINING A NORMAL GNN WITH NO RELATIONAL INFO NET

In [17]:
model_name = 'GNN_no_rel'
model = GNN_no_rel(5, edge_features, hidden_features, out_features, num_convolution_blocks=3, pooling='add').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)
train, valid, test = get_qm9("data/qm9",
                             device=device,
                             LABEL_INDEX=LABEL_INDEX,
                             transform=None)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=32, shuffle=False)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

In [18]:
gnn_no_rel_results = train_model(model, model_name, epochs, train_loader, valid_loader, test_loader, optimizer, criterion, scheduler, device, LABEL_INDEX, Z_ONE_HOT_DIM)

Total number of parameters: 277057


  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 0, Loss: 0.34073594618916514, Valid Loss: 0.26796615189804246


...

  0%|          | 0/3125 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch: 29, Loss: 1.61127240807582e-05, Valid Loss: 0.00044732314751880724


  0%|          | 0/3125 [00:00<?, ?it/s]

KeyboardInterrupt: 

# TRAINING A GNN WITH EDGE FEATURES

In [12]:
# create a fractal net and train it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'GNN'
model = GNN(n_node_features = Z_ONE_HOT_DIM,
            n_edge_features=EDGE_ATTR_DIM,
            n_hidden=64,
            n_output=out_features,
            num_convolution_blocks=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=3, verbose=True)
train, valid, test = get_qm9("data/qm9",
                             device=device,
                             LABEL_INDEX=LABEL_INDEX,
                             transform=None)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=32, shuffle=False)

/home/tin/Documents/GitHub/FractalMessagePassing/train.py:23: UserWarning: Using non-standard permutation since permute.pt does not exist.
  warn("Using non-standard permutation since permute.pt does not exist.")


In [13]:
gnn_results = train_model(model, model_name, epochs, train_loader, valid_loader, test_loader, optimizer, criterion, scheduler, device, LABEL_INDEX, Z_ONE_HOT_DIM)

Total number of parameters: 227329


  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

IndexError: too many indices for tensor of dimension 2

# PLOTTING LOSS

In [ ]:
# plot loss
# IGNORE FOR NOW #
import matplotlib.pyplot as plt
# plot train loss on same plot for different runs
plt.plot(fractalnetshared_results['train_loss'], label='FractalNetShared')
plt.plot(fractalnet_results['train_loss'], label='FractalNet')
plt.plot(gnn_no_rel_results['train_loss'], label='GNN_no_rel')
plt.plot(gnn_results['train_loss'], label='GNN')
plt.plot(no_subnode_results['train_loss'], label='No Subnodes')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# plot validation loss on same plot for different runs
plt.plot(fractalnetshared_results['valid_loss'], label='FractalNetShared')
plt.plot(fractalnet_results['valid_loss'], label='FractalNet')
plt.plot(gnn_no_rel_results['valid_loss'], label='GNN_no_rel')
plt.plot(gnn_results['valid_loss'], label='GNN')
plt.plot(no_subnode_results['valid_loss'], label='No Subnodes')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [21]:
# print final test losses of all models
#print('FractalNetShared Test Loss: ', fractalnetshared_results['test_loss'])
print('FractalNet Test Loss: ', fractalnet_results['test_loss'])
print('GNN_no_rel Test Loss: ', gnn_no_rel_results['test_loss'])
#print('GNN Test Loss: ', gnn_results['test_loss'])
print('No Subnodes Test Loss: ', no_subnode_results['test_loss'])

FractalNet Test Loss:  0.0005467538204020275
GNN_no_rel Test Loss:  1.4065201867546444e-05
No Subnodes Test Loss:  3.3362182585271106e-06
